<a href="https://colab.research.google.com/github/wjleece/AI-Agents/blob/main/AI_Agents_w_Evals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
%pip install anthropic
%pip install openai
%pip install -q -U google-generativeai
%pip install fuzzywuzzy

In [48]:
import anthropic
import google.generativeai as gemini
import re
import json
import time
from google.colab import userdata
from openai import OpenAI
from datetime import datetime
from typing import Dict, List, Any, Optional, Union, Tuple
from fuzzywuzzy import process, fuzz


ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

anthropic_client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)
openai_client = OpenAI(api_key=OPENAI_API_KEY)
gemini.configure(api_key=GOOGLE_API_KEY)

ANTHROPIC_MODEL_NAME = "claude-3-5-sonnet-latest"
OPENAI_MODEL_NAME = "gpt-4.1"
EVAL_MODEL_NAME = "gemini-2.5-pro-preview-05-06"

In [49]:
worker_system_prompt = """
You are a helpful customer service assistant for an e-commerce system.

When responding to the user, use the conversation context to maintain continuity.
- If a user refers to "my order" or similar, use the context to determine which order they're talking about.
- If they mention "that product" or use other references, check the context to determine what they're referring to.
- Always prioritize recent context over older context when resolving references.

The conversation context will be provided to you with each message. This includes:
- Previous questions and answers
- Recently viewed customers, products, and orders
- Recent actions taken (like creating orders, updating products, etc.)

BEHAVIOR FOR TARGETED REQUESTS:
If the user's query explicitly names the *other* AI assistant for a task (e.g., "OpenAI, do X" when you are Anthropic, or "Anthropic, do Y" when you are OpenAI), you MUST follow these steps:
1. Identify that the request is specifically for the other assistant.
2. Your *only* action should be to output a brief, polite acknowledgment. For example:
   - "Understood. I'll let OpenAI handle that."
   - "Okay, that request is for Anthropic."
   - Or simply: "Acknowledged."
3. You MUST NOT call any tools or attempt to perform the core task mentioned in the user's query. Your role in this specific instance is to defer.
Failure to defer when the other agent is explicitly named for a task will be considered incorrect behavior.

Keep responses friendly, concise, and helpful. If you're not sure what a user is referring to, ask for clarification.
"""

evaluator_system_prompt = """
You are an impartial evaluator assessing the quality of responses from two AI assistants (Anthropic Claude and OpenAI GPT) to customer service queries.

For each interaction, evaluate both responses based on:
1. Accuracy: How correct and factual is the response based on the available information?
2. Efficiency: Did the assistant get to the correct answer with minimal clarifying questions?
3. Context Awareness: Did the assistant correctly use the conversation context to understand references?
4. Helpfulness: How well did the assistant address the user's needs?

Score each response on a scale of 1-10 for each criterion, and provide an overall score.

If you identify ambiguity in the user's query that neither assistant could reasonably resolve without additional information:
1. ALWAYS begin your clarification request with the exact phrase "CLARIFICATION NEEDED:" followed by a specific question
2. Format your request clearly and precisely as "CLARIFICATION NEEDED: [your specific question here]"
3. Make your question answerable with a straightforward response
4. If multiple clarifications are needed, number them clearly

After receiving human clarification, continue your evaluation incorporating this new information.
Store this feedback as a "learning" so similar situations can be handled better in the future.

If multiple data stores are provided representing the state after each assistant's actions, you will be asked to compare them for consistency as a final step and comment on whether this comparison affects your initial scoring.

For testing purposes, you may be asked to identify which model you are. You should realize that type of question likely comes from
a human user and not from an AI assistant. Therefore you should properly identify yourself by stating which model you are, and,
if specifically asked, your key tasks.
"""

In [50]:
# The GenerativeModel instance for evaluation will be created with the system instruction.
eval_model_instance = gemini.GenerativeModel(
    model_name=EVAL_MODEL_NAME,
    system_instruction=evaluator_system_prompt
)

In [51]:
# Cell 5: Global Data Stores (Initial data - will be managed by the Storage class instance)
# These are initial values. The Storage class will manage them.
initial_customers = {
    "C1": {"name": "John Doe", "email": "john@example.com", "phone": "123-456-7890"},
    "C2": {"name": "Jane Smith", "email": "jane@example.com", "phone": "987-654-3210"}
}

initial_products = {
    "P1": {"name": "Widget A", "description": "A simple widget. Very compact.", "price": 19.99, "inventory_count": 999},
    "P2": {"name": "Gadget B", "description": "A powerful gadget. It spins.", "price": 49.99, "inventory_count": 200},
    "P3": {"name": "Perplexinator", "description": "A perplexing perfunctator", "price": 79.99, "inventory_count": 1483}
}

initial_orders = {
    "O1": {"id": "O1", "product_id": "P1", "product_name": "Widget A", "quantity": 2, "price": 19.99, "status": "Shipped"},
    "O2": {"id": "O2", "product_id": "P2", "product_name": "Gadget B", "quantity": 1, "price": 49.99, "status": "Processing"}
}


In [52]:
#Knowledge base and Global Tools Placeholder
human_feedback_learnings = {}
tools_schemas_list = []

In [53]:
# Standalone Anthropic Completion Function (for basic tests)
def get_completion_anthropic_standalone(prompt: str):
    message = anthropic_client.messages.create(
        model=ANTHROPIC_MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=worker_system_prompt,
        tools=tools_schemas_list,
        messages=[
          {"role": "user", "content": prompt}
        ]
    )
    return message.content[0].text

In [54]:
prompt_test_anthropic = "Hey there, which AI model do you use for answering questions?"
print(f"Anthropic Standalone Test: {get_completion_anthropic_standalone(prompt_test_anthropic)}")

Anthropic Standalone Test: I am Claude, created by Anthropic. I aim to be direct and honest about my identity while focusing on providing helpful customer service assistance for the e-commerce system.


In [55]:
def get_completion_openai_standalone(prompt: str):
    response = openai_client.chat.completions.create(
        model=OPENAI_MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        tools=tools_schemas_list,
        messages=[
            {"role": "system", "content": worker_system_prompt},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

In [56]:
prompt_test_openai = "Hey there, which AI model do you use for answering questions?"
print(f"OpenAI Standalone Test: {get_completion_openai_standalone(prompt_test_openai)}")

OpenAI Standalone Test: Hello! I’m powered by Anthropic’s AI technology to assist you with your questions and help you with your e-commerce needs. If you have any specific questions or need help with your orders, just let me know!


In [57]:
def get_completion_eval_standalone(prompt: str):
    # Uses the eval_model_instance defined in Cell 4 which has the system prompt
        response = eval_model_instance.generate_content(prompt)
        return response.text

In [58]:
prompt_test_eval = "Hey there, can you tell me which AI you are and what your key tasks are?"
print(f"Gemini Eval Standalone Test:\n{get_completion_eval_standalone(prompt_test_eval)}")

Gemini Eval Standalone Test:
I am a large language model, trained by Google. My key tasks are to act as an impartial evaluator assessing the quality of responses from two AI assistants to customer service queries. I evaluate these responses based on accuracy, efficiency, context awareness, and helpfulness, providing scores and feedback for each.


In [59]:
# Storage Class Definition
class Storage:
    """Storage class for global e-commerce data access"""
    def __init__(self):
        # Each Storage instance gets its own copy of the initial data
        self.customers = initial_customers.copy()
        self.products = initial_products.copy()
        self.orders = initial_orders.copy()
        # Note: human_feedback_learnings is still a shared global dictionary
        self.human_feedback_learnings = human_feedback_learnings

# This global instance is for legacy/standalone tool testing if any.
# The DualAgentEvaluator will create its own instances for Anthropic and OpenAI.
storage_global_for_standalone_tests = Storage()
print("Storage class defined. Note: DualAgentEvaluator will use its own Storage instances.")

Storage class defined. Note: DualAgentEvaluator will use its own Storage instances.


In [60]:
#Definitive list of tool schemas.
tools_schemas_list = [
    {
        "name": "create_customer",
        "description": "Adds a new customer to the database. Includes customer name, email, and (optional) phone number.",
        "input_schema": {
            "type": "object",
            "properties": {
                "name": {"type": "string", "description": "The name of the customer."},
                "email": {"type": "string", "description": "The email address of the customer."},
                "phone": {"type": "string", "description": "The phone number of the customer (optional)."}
            },
            "required": ["name", "email"]
        }
    },
    {
        "name": "get_customer_info",
        "description": "Retrieves customer information based on their customer ID. Returns the customer's name, email, and (optional) phone number.",
        "input_schema": {
            "type": "object",
            "properties": {
                "customer_id": {"type": "string", "description": "The unique identifier for the customer."}
            },
            "required": ["customer_id"]
        }
    },
    {
        "name": "create_product",
        "description": "Adds a new product to the product database. Includes name, description, price, and initial inventory count.",
        "input_schema": {
            "type": "object",
            "properties": {
                "name": {"type": "string", "description": "The name of the product."},
                "description": {"type": "string", "description": "A description of the product."},
                "price": {"type": "number", "description": "The price of the product."},
                "inventory_count": {"type": "integer", "description": "The amount of the product that is currently in inventory."}
            },
            "required": ["name", "description", "price", "inventory_count"]
        }
    },
    {
        "name": "update_product",
        "description": "Updates an existing product with new information. Only fields that are provided will be updated; other fields remain unchanged.",
        "input_schema": {
            "type": "object",
            "properties": {
                "product_id": {"type": "string", "description": "The unique identifier for the product to update."},
                "name": {"type": "string", "description": "The new name for the product (optional)."},
                "description": {"type": "string", "description": "The new description for the product (optional)."},
                "price": {"type": "number", "description": "The new price for the product (optional)."},
                "inventory_count": {"type": "integer", "description": "The new inventory count for the product (optional)."}
            },
            "required": ["product_id"]
        }
    },
    {
        "name": "get_product_info",
        "description": "Retrieves product information based on product ID or product name (with fuzzy matching for misspellings). Returns product details including name, description, price, and inventory count.",
        "input_schema": {
            "type": "object",
            "properties": {
                "product_id_or_name": {"type": "string", "description": "The product ID or name (can be approximate)."}
            },
            "required": ["product_id_or_name"]
        }
    },
    {
        "name": "list_all_products",
        "description": "Lists all available products in the inventory.",
        "input_schema": { "type": "object", "properties": {}, "required": [] }
    },
    {
        "name": "create_order",
        "description": "Creates an order using the product's current price. If requested quantity exceeds available inventory, no order is created and available quantity is returned. Orders can only be created for products that are in stock. Supports specifying products by either ID or name with fuzzy matching for misspellings.",
        "input_schema": {
            "type": "object",
            "properties": {
                "product_id_or_name": {"type": "string", "description": "The ID or name of the product to order (supports fuzzy matching)."},
                "quantity": {"type": "integer", "description": "The quantity of the product in the order."},
                "status": {"type": "string", "description": "The initial status of the order (e.g., 'Processing', 'Shipped')."}
            },
            "required": ["product_id_or_name", "quantity", "status"]
        }
    },
    {
        "name": "get_order_details",
        "description": "Retrieves the details of a specific order based on the order ID. Returns the order ID, product name, quantity, price, and order status.",
        "input_schema": {
            "type": "object",
            "properties": {
                "order_id": {"type": "string", "description": "The unique identifier for the order."}
            },
            "required": ["order_id"]
        }
    },
    {
        "name": "update_order_status",
        "description": "Updates the status of an order and adjusts inventory accordingly. Changing to \"Shipped\" decreases inventory. Changing to \"Returned\" or \"Canceled\" from \"Shipped\" increases inventory. Status can be \"Processing\", \"Shipped\", \"Delivered\", \"Returned\", or \"Canceled\".",
        "input_schema": {
            "type": "object",
            "properties": {
                "order_id": {"type": "string", "description": "The unique identifier for the order."},
                "new_status": {
                    "type": "string",
                    "description": "The new status to set for the order.",
                    "enum": ["Processing", "Shipped", "Delivered", "Returned", "Canceled"]
                }
            },
            "required": ["order_id", "new_status"]
        }
    }
]
print(f"Defined {len(tools_schemas_list)} tool schemas.")


Defined 9 tool schemas.


In [61]:
# Tool Function Definitions
# These tool functions now accept a 'current_storage' argument to operate on a specific Storage instance.

# Customer functions
def create_customer(current_storage: Storage, name: str, email: str, phone: Optional[str] = None) -> Dict[str, Any]:
    """Creates a new customer and adds them to the customer database."""
    new_id = f"C{len(current_storage.customers) + 1}"
    current_storage.customers[new_id] = {"name": name, "email": email, "phone": phone}
    print(f"[Tool Executed] create_customer: ID {new_id}, Name: {name} (in {type(current_storage).__name__})")
    return {"status": "success", "customer_id": new_id, "customer": current_storage.customers[new_id]}

def get_customer_info(current_storage: Storage, customer_id: str) -> Dict[str, Any]:
    """Retrieves information about a customer based on their ID."""
    customer = current_storage.customers.get(customer_id)
    if customer:
        print(f"[Tool Executed] get_customer_info: ID {customer_id} found (in {type(current_storage).__name__}).")
        return {"status": "success", "customer_id": customer_id, "customer": customer}
    print(f"[Tool Executed] get_customer_info: ID {customer_id} not found (in {type(current_storage).__name__}).")
    return {"status": "error", "message": "Customer not found"}

# Product functions
def create_product(current_storage: Storage, name: str, description: str, price: float, inventory_count: int) -> Dict[str, Any]:
    """Creates a new product and adds it to the product database."""
    new_id = f"P{len(current_storage.products) + 1}"
    current_storage.products[new_id] = {
        "name": name,
        "description": description,
        "price": float(price),
        "inventory_count": int(inventory_count)
    }
    print(f"[Tool Executed] create_product: ID {new_id}, Name: {name} (in {type(current_storage).__name__})")
    return {"status": "success", "product_id": new_id, "product": current_storage.products[new_id]}

def update_product(current_storage: Storage, product_id: str, name: Optional[str] = None, description: Optional[str] = None,
                   price: Optional[float] = None, inventory_count: Optional[int] = None) -> Dict[str, Any]:
    """Updates a product with the provided parameters."""
    if product_id not in current_storage.products:
        print(f"[Tool Executed] update_product: ID {product_id} not found (in {type(current_storage).__name__}).")
        return {"status": "error", "message": f"Product {product_id} not found"}

    product = current_storage.products[product_id]
    updated_fields = []

    if name is not None:
        product["name"] = name
        updated_fields.append("name")
    if description is not None:
        product["description"] = description
        updated_fields.append("description")
    if price is not None:
        product["price"] = float(price)
        updated_fields.append("price")
    if inventory_count is not None:
        product["inventory_count"] = int(inventory_count)
        updated_fields.append("inventory_count")

    if not updated_fields:
        print(f"[Tool Executed] update_product: ID {product_id}, no fields updated (in {type(current_storage).__name__}).")
        return {"status": "warning", "message": "No fields were updated.", "product": product}

    print(f"[Tool Executed] update_product: ID {product_id}, Updated fields: {', '.join(updated_fields)} (in {type(current_storage).__name__})")
    return {
        "status": "success",
        "message": f"Product {product_id} updated. Fields: {', '.join(updated_fields)}",
        "product_id": product_id,
        "updated_fields": updated_fields,
        "product": product
    }

def find_product_by_name(current_storage: Storage, product_name: str, min_similarity: int = 70) -> Tuple[Optional[str], Optional[Dict[str, Any]]]:
    """Find a product by name using fuzzy string matching."""
    if not product_name: return None, None

    name_id_list = [(p_data["name"], p_id) for p_id, p_data in current_storage.products.items()]
    if not name_id_list: return None, None

    best_match_name_score = process.extractOne(
        product_name,
        [item[0] for item in name_id_list],
        scorer=fuzz.token_sort_ratio
    )

    if best_match_name_score and best_match_name_score[1] >= min_similarity:
        matched_name = best_match_name_score[0]
        for name, pid_val in name_id_list:
            if name == matched_name:
                print(f"[Tool Helper] find_product_by_name: Matched '{product_name}' to '{matched_name}' (ID: {pid_val}) with score {best_match_name_score[1]} (in {type(current_storage).__name__})")
                return pid_val, current_storage.products[pid_val]

    print(f"[Tool Helper] find_product_by_name: No good match for '{product_name}' (min_similarity: {min_similarity}, Best match: {best_match_name_score}) (in {type(current_storage).__name__})")
    return None, None


def get_product_id(current_storage: Storage, product_identifier: str) -> Optional[str]:
    """Get product ID either directly or by fuzzy matching the name."""
    if product_identifier in current_storage.products:
        return product_identifier
    product_id, _ = find_product_by_name(current_storage, product_identifier)
    return product_id

def get_product_info(current_storage: Storage, product_id_or_name: str) -> Dict[str, Any]:
    """Get information about a product by its ID or name."""
    if product_id_or_name in current_storage.products:
        product = current_storage.products[product_id_or_name]
        print(f"[Tool Executed] get_product_info: Found by ID '{product_id_or_name}' (in {type(current_storage).__name__}).")
        return {"status": "success", "product_id": product_id_or_name, "product": product}

    # Use the modified find_product_by_name that takes current_storage
    product_id_found, product_data = find_product_by_name(current_storage, product_id_or_name)
    if product_id_found and product_data:
        print(f"[Tool Executed] get_product_info: Found by name (fuzzy) '{product_id_or_name}' as ID '{product_id_found}' (in {type(current_storage).__name__}).")
        return {"status": "success", "message": f"Found product matching '{product_id_or_name}'", "product_id": product_id_found, "product": product_data}

    print(f"[Tool Executed] get_product_info: No product found for '{product_id_or_name}' (in {type(current_storage).__name__}).")
    return {"status": "error", "message": f"No product found matching '{product_id_or_name}'"}


def list_all_products(current_storage: Storage) -> Dict[str, Any]:
    """List all available products in the inventory."""
    print(f"[Tool Executed] list_all_products: Found {len(current_storage.products)} products (in {type(current_storage).__name__}).")
    return {"status": "success", "count": len(current_storage.products), "products": dict(current_storage.products)}

# Order functions
def create_order(current_storage: Storage, product_id_or_name: str, quantity: int, status: str) -> Dict[str, Any]:
    """Creates an order using the product's stored price."""
    actual_product_id = get_product_id(current_storage, product_id_or_name) # Pass current_storage

    if not actual_product_id:
        print(f"[Tool Executed] create_order: Product '{product_id_or_name}' not found (in {type(current_storage).__name__}).")
        return {"status": "error", "message": f"Product '{product_id_or_name}' not found."}

    product = current_storage.products[actual_product_id]
    price = product["price"]

    if product["inventory_count"] == 0:
        print(f"[Tool Executed] create_order: Product ID {actual_product_id} is out of stock (in {type(current_storage).__name__}).")
        return {"status": "error", "message": f"{product['name']} is out of stock."}
    if quantity <= 0:
        print(f"[Tool Executed] create_order: Quantity must be positive. Requested: {quantity} (in {type(current_storage).__name__})")
        return {"status": "error", "message": "Quantity must be a positive number."}
    if quantity > product["inventory_count"]:
        print(f"[Tool Executed] create_order: Insufficient inventory for {product['name']} (ID: {actual_product_id}). Available: {product['inventory_count']}, Requested: {quantity} (in {type(current_storage).__name__})")
        return {
            "status": "partial_availability",
            "message": f"Insufficient inventory. Only {product['inventory_count']} units of {product['name']} are available.",
            "available_quantity": product["inventory_count"],
            "requested_quantity": quantity,
            "product_name": product['name']
        }

    if status == "Shipped":
        product["inventory_count"] -= quantity
        print(f"[Tool Executed] create_order: Inventory for {product['name']} (ID: {actual_product_id}) reduced by {quantity} due to 'Shipped' status on creation (in {type(current_storage).__name__}).")

    new_id = f"O{len(current_storage.orders) + 1}"
    current_storage.orders[new_id] = {
        "id": new_id,
        "product_id": actual_product_id,
        "product_name": product["name"],
        "quantity": quantity,
        "price": price,
        "status": status
    }
    print(f"[Tool Executed] create_order: Order {new_id} created for {quantity} of {product['name']} (ID: {actual_product_id}). Status: {status}. Remaining inv: {product['inventory_count']} (in {type(current_storage).__name__})")
    return {
        "status": "success",
        "order_id": new_id,
        "order_details": current_storage.orders[new_id],
        "remaining_inventory": product["inventory_count"]
    }

def get_order_details(current_storage: Storage, order_id: str) -> Dict[str, Any]:
    """Get details of a specific order."""
    order = current_storage.orders.get(order_id)
    if order:
        print(f"[Tool Executed] get_order_details: Order {order_id} found (in {type(current_storage).__name__}).")
        return {"status": "success", "order_id": order_id, "order_details": dict(order)}
    print(f"[Tool Executed] get_order_details: Order {order_id} not found (in {type(current_storage).__name__}).")
    return {"status": "error", "message": "Order not found"}

def update_order_status(current_storage: Storage, order_id: str, new_status: str) -> Dict[str, Any]:
    """Updates the status of an order and adjusts inventory accordingly."""
    if order_id not in current_storage.orders:
        print(f"[Tool Executed] update_order_status: Order {order_id} not found (in {type(current_storage).__name__}).")
        return {"status": "error", "message": "Order not found"}

    order = current_storage.orders[order_id]
    old_status = order["status"]
    product_id = order["product_id"]
    quantity = order["quantity"]

    if old_status == new_status:
        print(f"[Tool Executed] update_order_status: Order {order_id} status unchanged ({old_status}) (in {type(current_storage).__name__}).")
        return {"status": "unchanged", "message": f"Order {order_id} status is already {old_status}", "order_details": dict(order)}

    inventory_adjusted = False
    current_inventory_val = "unknown" # Default if product not found (should not happen if order is valid)

    if product_id in current_storage.products:
        product = current_storage.products[product_id]
        current_inventory_val = product["inventory_count"]

        if new_status == "Shipped" and old_status not in ["Shipped", "Delivered"]:
            if current_inventory_val < quantity:
                print(f"[Tool Executed] update_order_status: Insufficient inventory to ship order {order_id}. Have {current_inventory_val}, need {quantity} (in {type(current_storage).__name__}).")
                return {"status": "error", "message": f"Insufficient inventory to ship. Available: {current_inventory_val}, Required: {quantity}"}
            product["inventory_count"] -= quantity
            inventory_adjusted = True
            current_inventory_val = product["inventory_count"]
            print(f"[Tool Executed] update_order_status: Order {order_id} Shipped. Inv for {product_id} reduced by {quantity} to {current_inventory_val} (in {type(current_storage).__name__}).")
        elif new_status in ["Returned", "Canceled"] and old_status in ["Shipped", "Delivered"]:
            product["inventory_count"] += quantity
            inventory_adjusted = True
            current_inventory_val = product["inventory_count"]
            print(f"[Tool Executed] update_order_status: Order {order_id} {new_status}. Inv for {product_id} increased by {quantity} to {current_inventory_val} (in {type(current_storage).__name__}).")
    else:
        print(f"[Tool Executed] update_order_status: Product {product_id} for order {order_id} not found for inventory adjustment (in {type(current_storage).__name__}).")

    order["status"] = new_status
    print(f"[Tool Executed] update_order_status: Order {order_id} status updated from {old_status} to {new_status} (in {type(current_storage).__name__}).")
    return {
        "status": "success",
        "message": f"Order {order_id} status updated from {old_status} to {new_status}.",
        "order_id": order_id,
        "product_id": product_id,
        "old_status": old_status,
        "new_status": new_status,
        "inventory_adjusted": inventory_adjusted,
        "current_inventory": current_inventory_val,
        "order_details": dict(order)
    }

print("Tool functions defined.")

Tool functions defined.


In [62]:
class ConversationContext:
    def __init__(self):
        self.messages: List[Dict[str, Any]] = []
        self.context_data: Dict[str, Any] = {
            "customers": {}, "products": {}, "orders": {}, "last_action": None
        }
        self.session_start_time = datetime.now()

    def add_user_message(self, message: str) -> None:
        self.messages.append({"role": "user", "content": message})

    def add_assistant_message(self, message_content: Union[str, List[Dict[str, Any]]]) -> None:
        self.messages.append({"role": "assistant", "content": message_content})

    def update_entity_in_context(self, entity_type: str, entity_id: str, data: Any) -> None:
        if entity_type in self.context_data:
            self.context_data[entity_type][entity_id] = data # Store the actual data
            print(f"[Context Updated] Entity: {entity_type}, ID: {entity_id}, Data (type): {type(data)}")

    def set_last_action(self, action_type: str, action_details: Any) -> None:
        self.context_data["last_action"] = {
            "type": action_type,
            "details": action_details,
            "timestamp": datetime.now().isoformat()
        }
        print(f"[Context Updated] Last Action: {action_type}, Details: {json.dumps(action_details, default=str)}")


    def get_full_conversation_for_api(self) -> List[Dict[str, Any]]:
        return self.messages.copy()

    def get_context_summary(self) -> str:
        summary_parts = []
        if self.context_data["customers"]:
            customers_str = ", ".join([f"ID: {cid} (Name: {c.get('name', 'N/A') if isinstance(c, dict) else 'N/A'})" for cid, c in self.context_data["customers"].items()])
            summary_parts.append(f"Recent customers: {customers_str}")
        if self.context_data["products"]:
            products_str = ", ".join([f"ID: {pid} (Name: {p.get('name', 'N/A') if isinstance(p, dict) else 'N/A'})" for pid, p in self.context_data["products"].items()])
            summary_parts.append(f"Recent products: {products_str}")
        if self.context_data["orders"]:
            orders_str = ", ".join([f"ID: {oid} (Product: {o.get('product_name', 'N/A') if isinstance(o, dict) else 'N/A'}, Status: {o.get('status', 'N/A') if isinstance(o, dict) else 'N/A'})" for oid, o in self.context_data["orders"].items()])
            summary_parts.append(f"Recent orders: {orders_str}")

        last_action = self.context_data["last_action"]
        if last_action:
            action_type = last_action['type']
            action_details_summary = "..." # Default summary
            if isinstance(last_action.get('details'), dict):
                action_input = last_action['details'].get('input', {})
                action_result_status = last_action['details'].get('result', {}).get('status')
                action_details_summary = f"Input: {action_input}, Result Status: {action_result_status}"
                if action_result_status == "success":
                    if "order_id" in last_action['details'].get('result', {}):
                         action_details_summary += f", OrderID: {last_action['details']['result']['order_id']}"
                    elif "product_id" in last_action['details'].get('result', {}):
                         action_details_summary += f", ProductID: {last_action['details']['result']['product_id']}"


            summary_parts.append(f"Last action: {action_type} at {last_action['timestamp']} ({action_details_summary})")

        if not summary_parts: return "No specific context items set yet."
        return "\n".join(summary_parts)

    def clear(self) -> None:
        self.messages = []
        self.context_data = {"customers": {}, "products": {}, "orders": {}, "last_action": None}
        self.session_start_time = datetime.now()
        print("[Context Cleared]")

print("ConversationContext class defined.")


ConversationContext class defined.


In [63]:
# Cell 19: DualAgentEvaluator Class Definition
class DualAgentEvaluator:
    def __init__(self):
        self.conversation_context = ConversationContext()
        self.evaluation_results = []

        # Initialize separate storage instances for each agent
        self.anthropic_storage = Storage()
        self.openai_storage = Storage()
        print("DualAgentEvaluator initialized with separate Storage for Anthropic and OpenAI.")

        self.anthropic_tools_schemas = tools_schemas_list
        self.openai_tools_formatted = []
        if tools_schemas_list:
            self.openai_tools_formatted = [
                {"type": "function", "function": tool_def} for tool_def in tools_schemas_list
            ] # OpenAI new format

        self.available_tool_functions = {
            "create_customer": create_customer, "get_customer_info": get_customer_info,
            "create_product": create_product, "update_product": update_product,
            "get_product_info": get_product_info, "list_all_products": list_all_products,
            "create_order": create_order, "get_order_details": get_order_details,
            "update_order_status": update_order_status,
        }
        self.human_feedback_learnings = human_feedback_learnings # Global dict
        print(f"DualAgentEvaluator initialized. OpenAI tools formatted ({len(self.openai_tools_formatted)} tools).")


    def _update_context_from_tool_results(self, tool_name: str, tool_input: Dict, tool_result: Dict, agent_name: str):
        """Helper to update conversation context based on tool results. Agent name for logging."""
        if not isinstance(tool_result, dict):
            print(f"[Context Update Error] Tool result for {tool_name} ({agent_name}) is not a dict: {tool_result}")
            self.conversation_context.set_last_action(f"{tool_name}_{agent_name}", {"input": tool_input, "result": {"status": "error", "message": "Tool result was not a dictionary."}})
            return

        # The conversation context primarily tracks the flow and entities mentioned.
        # The actual data change has already happened in the specific agent's storage.
        # We update the shared context with the IDs and data for general awareness.
        if tool_result.get("status") == "success":
            if "customer_id" in tool_result and "customer" in tool_result and isinstance(tool_result["customer"], dict):
                self.conversation_context.update_entity_in_context("customers", tool_result["customer_id"], tool_result["customer"])
            elif "product_id" in tool_result and "product" in tool_result and isinstance(tool_result["product"], dict):
                self.conversation_context.update_entity_in_context("products", tool_result["product_id"], tool_result["product"])
            elif "order_id" in tool_result and "order_details" in tool_result and isinstance(tool_result["order_details"], dict):
                self.conversation_context.update_entity_in_context("orders", tool_result["order_id"], tool_result["order_details"])
            elif tool_name == "list_all_products" and "products" in tool_result and isinstance(tool_result["products"], dict):
                 for pid, pdata in tool_result["products"].items():
                     self.conversation_context.update_entity_in_context("products", pid, pdata)

        # Tag the last action with the agent name for clarity
        self.conversation_context.set_last_action(f"{tool_name}_{agent_name}", {"input": tool_input, "result": tool_result})


    def process_tool_call(self, tool_name: str, tool_input: Dict[str, Any], target_storage_instance: Storage) -> Dict[str, Any]:
        print(f"--- [Tool Dispatcher] Attempting tool: {tool_name} with input: {json.dumps(tool_input, default=str)} for storage: {type(target_storage_instance).__name__} ---")
        if tool_name in self.available_tool_functions:
            function_to_call = self.available_tool_functions[tool_name]
            try:
                # Pass the target_storage_instance to the tool function
                result = function_to_call(target_storage_instance, **tool_input)
                print(f"--- [Tool Dispatcher] Result for {tool_name} on {type(target_storage_instance).__name__}: {json.dumps(result, indent=2, default=str)} ---")
                return result
            except TypeError as te:
                print(f"--- [Tool Dispatcher] TypeError for {tool_name} on {type(target_storage_instance).__name__}: {te}. Input: {tool_input} ---")
                return {"status": "error", "message": f"TypeError calling {tool_name}: {str(te)}. Check arguments."}
            except Exception as e:
                print(f"--- [Tool Dispatcher] Exception for {tool_name} on {type(target_storage_instance).__name__}: {e} ---")
                return {"status": "error", "message": f"Error executing {tool_name}: {str(e)}"}
        else:
            print(f"--- [Tool Dispatcher] Tool {tool_name} not found. ---")
            return {"status": "error", "message": f"Tool {tool_name} not found."}

    def get_anthropic_response(self, current_worker_system_prompt: str, conversation_history: List[Dict[str, Any]]) -> str:
        messages_for_api = conversation_history.copy()
        try:
            for i in range(5): # Max 5 tool iterations
                system_prompt_snippet = current_worker_system_prompt[:60].replace('\n', ' ')
                print(f"\nAnthropic API Call #{i+1}. System: '{system_prompt_snippet}...', Messages count: {len(messages_for_api)}")
                if messages_for_api: print(f"Last message role: {messages_for_api[-1]['role']}")

                response = anthropic_client.messages.create(
                    model=ANTHROPIC_MODEL_NAME, max_tokens=4000,
                    system=current_worker_system_prompt,
                    tools=self.anthropic_tools_schemas,
                    messages=messages_for_api
                )

                assistant_response_blocks = response.content
                messages_for_api.append({"role": "assistant", "content": assistant_response_blocks})

                tool_calls_to_process = [block for block in assistant_response_blocks if block.type == "tool_use"]
                text_blocks = [block.text for block in assistant_response_blocks if block.type == "text"]

                if not tool_calls_to_process:
                    final_text = " ".join(text_blocks).strip()
                    print(f"Anthropic Final Text (no tool use this turn): {final_text}")
                    return final_text if final_text else "No text content in final Anthropic response."

                tool_results_for_next_call = []
                for tool_use_block in tool_calls_to_process:
                    tool_name, tool_input, tool_use_id = tool_use_block.name, tool_use_block.input, tool_use_block.id
                    print(f"Anthropic Tool Call: {tool_name}, Input: {tool_input}")
                    # Use self.anthropic_storage for Anthropic's tool calls
                    tool_result_data = self.process_tool_call(tool_name, tool_input, self.anthropic_storage)
                    self._update_context_from_tool_results(tool_name, tool_input, tool_result_data, "Anthropic")

                    tool_results_for_next_call.append({
                        "type": "tool_result", "tool_use_id": tool_use_id,
                        "content": json.dumps(tool_result_data)
                    })
                messages_for_api.append({"role": "user", "content": tool_results_for_next_call})
            return "Max tool iterations reached for Anthropic."
        except Exception as e:
            print(f"Error in get_anthropic_response: {str(e)}")
            import traceback; traceback.print_exc()
            return f"Error getting Anthropic response: {str(e)}"

    def get_openai_response(self, current_worker_system_prompt: str, conversation_history: List[Dict[str, Any]]) -> str:
        messages_for_api = [{"role": "system", "content": current_worker_system_prompt}]
        # Filter conversation history for OpenAI.
        # This naive approach just takes user messages and text-based assistant messages.
        # A more robust solution would convert message formats or maintain separate histories.
        for msg in conversation_history:
            if msg["role"] == "user":
                messages_for_api.append(msg)
            elif msg["role"] == "assistant":
                if isinstance(msg["content"], str): # Simple text response
                     messages_for_api.append(msg)
                # Potentially handle OpenAI's own tool call format if it were in shared history
                elif isinstance(msg["content"], dict) and "tool_calls" in msg["content"]:
                     messages_for_api.append(msg)
                # else: skip complex Anthropic blocks for OpenAI for now

        try:
            for i in range(5): # Max 5 tool iterations
                print(f"\nOpenAI API Call #{i+1}. Messages count: {len(messages_for_api)}")
                if messages_for_api and isinstance(messages_for_api[-1], dict):
                    print(f"Last message role: {messages_for_api[-1].get('role', 'N/A')}")

                response = openai_client.chat.completions.create(
                    model=OPENAI_MODEL_NAME,
                    messages=messages_for_api,
                    tools=self.openai_tools_formatted,
                    tool_choice="auto"
                )
                response_message = response.choices[0].message
                messages_for_api.append(response_message.model_dump()) # Add assistant's turn

                if not response_message.tool_calls:
                    final_text = response_message.content if response_message.content else "No text content in final OpenAI response."
                    print(f"OpenAI Final Text (no tool use this turn): {final_text}")
                    return final_text

                tool_results_for_next_api_call = []
                for tool_call in response_message.tool_calls:
                    tool_name = tool_call.function.name
                    tool_input_str = tool_call.function.arguments
                    tool_call_id = tool_call.id
                    try:
                        tool_input = json.loads(tool_input_str)
                    except json.JSONDecodeError:
                        print(f"OpenAI Tool Call JSON Error for {tool_name}: {tool_input_str}")
                        tool_result_data = {"status": "error", "message": "Invalid JSON arguments from model."}
                    else:
                        print(f"OpenAI Tool Call: {tool_name}, Input: {tool_input}")
                        # Use self.openai_storage for OpenAI's tool calls
                        tool_result_data = self.process_tool_call(tool_name, tool_input, self.openai_storage)

                    self._update_context_from_tool_results(tool_name, tool_input, tool_result_data, "OpenAI")
                    tool_results_for_next_api_call.append({
                        "tool_call_id": tool_call_id, "role": "tool", "name": tool_name,
                        "content": json.dumps(tool_result_data)
                    })
                messages_for_api.extend(tool_results_for_next_api_call)
            return "Max tool iterations reached for OpenAI."
        except Exception as e:
            print(f"Error in get_openai_response: {str(e)}")
            import traceback; traceback.print_exc()
            return f"Error getting OpenAI response: {str(e)}"

    def process_user_request(self, user_message: str) -> Dict[str, Any]:
        print(f"\n\n{'='*60}\nUser Message: {user_message}\n{'='*60}")
        self.conversation_context.add_user_message(user_message)

        context_summary = self.conversation_context.get_context_summary()
        print(f"Current Context Summary for Models:\n{context_summary}\n{'-'*60}")

        relevant_learnings = self.check_relevant_learnings(user_message)
        learnings_for_prompt = ""
        if relevant_learnings:
            learnings_for_prompt = f"\n\nRelevant Past Learnings:\n{relevant_learnings}"
            print(f"Relevant Learnings for this turn:\n{relevant_learnings}")


        current_worker_prompt_with_context_and_learnings = f"{worker_system_prompt}\n\nConversation Context:\n{context_summary}{learnings_for_prompt}"

        # Get a clean copy of history for each LLM.
        # The LLM-specific methods will prepend their system messages.
        # The shared history in conversation_context might contain mixed formats.
        # For simplicity, we pass the full history, and each LLM method should ideally
        # adapt or filter it if strict format separation is needed.
        # For now, get_anthropic_response uses it as is, get_openai_response does some filtering.

        anthropic_history_for_call = self.conversation_context.get_full_conversation_for_api()
        anthropic_response_text = self.get_anthropic_response(current_worker_prompt_with_context_and_learnings, anthropic_history_for_call)
        self.conversation_context.add_assistant_message(f"[Anthropic Final Text]: {anthropic_response_text}") # Simplified for shared context


        openai_history_for_call = self.conversation_context.get_full_conversation_for_api() # Get fresh copy
        openai_response_text = self.get_openai_response(current_worker_prompt_with_context_and_learnings, openai_history_for_call)
        self.conversation_context.add_assistant_message(f"[OpenAI Final Text]: {openai_response_text}") # Simplified for shared context

        print(f"\n--- Anthropic Final Response Text ---\n{anthropic_response_text}")
        print(f"--- OpenAI Final Response Text ---\n{openai_response_text}")

        evaluation = self.evaluate_responses(user_message, anthropic_response_text, openai_response_text, context_summary, learnings_for_prompt)
        self.evaluation_results.append(evaluation)

        # After evaluation and potential human feedback within evaluate_responses,
        # ask for general learnings.
        try:
            human_general_learning = input("Do you want to add any general learnings from this turn? (Type your learning or 'skip'): ")
            if human_general_learning.lower() != 'skip' and human_general_learning.strip():
                self.store_learning(user_message, "General feedback post-evaluation", human_general_learning)
                print(f"General learning stored: '{human_general_learning}'")
        except EOFError:
            print("EOFError: Skipping general learning input (non-interactive).")


        return {
            "user_message": user_message,
            "anthropic_response": anthropic_response_text,
            "openai_response": openai_response_text,
            "evaluation": evaluation
        }

    def process_human_feedback_actions(self, feedback: str, target_storage_for_action: Optional[Storage]) -> str:
        action_result_summary = "No specific data action taken based on feedback."
        if not target_storage_for_action:
            action_result_summary = "Skipping data action: No target storage specified for feedback."
            print(f"[Human Feedback Action] {action_result_summary}")
            return action_result_summary

        # Order updates
        order_update_match = re.search(r"update\s+order\s+(\w+)\s+status\s+to\s+(\w+)", feedback, re.IGNORECASE)
        if order_update_match:
            order_id, new_status = order_update_match.groups()
            try:
                result = self.process_tool_call("update_order_status", {"order_id": order_id, "new_status": new_status}, target_storage_for_action)
                action_result_summary = f"Action executed on {type(target_storage_for_action).__name__}: Updated order {order_id} status to {new_status}. Result: {result.get('status', 'N/A')}"
                # Update context if successful
                if result.get("status") == "success" and "order_details" in result:
                     self.conversation_context.update_entity_in_context("orders", order_id, result["order_details"])
                     self.conversation_context.set_last_action(f"human_feedback_update_order_{type(target_storage_for_action).__name__}",
                                                              {"input": {"order_id": order_id, "new_status": new_status}, "result": result})

            except Exception as e:
                action_result_summary = f"Failed to update order {order_id} on {type(target_storage_for_action).__name__}: {str(e)}"
            print(f"[Human Feedback Action] {action_result_summary}")
            return action_result_summary

        # Product creation
        product_create_match = re.search(
            r"create\s+(?:new\s+)?product:\\s*(.*?),\\s*description:\\s*(.*?),\\s*price:\\s*(\\d+\\.?\\d*),\\s*inventory:\\s*(\\d+)",
            feedback, re.IGNORECASE
        )
        if product_create_match:
            name, desc, price, inventory = product_create_match.groups()
            try:
                tool_input = {"name": name.strip(), "description": desc.strip(), "price": float(price), "inventory_count": int(inventory)}
                result = self.process_tool_call("create_product", tool_input, target_storage_for_action)
                action_result_summary = f"Action executed on {type(target_storage_for_action).__name__}: Created product '{name}'. Result: {result.get('status', 'N/A')}"
                if result.get("status") == "success" and "product" in result:
                     self.conversation_context.update_entity_in_context("products", result["product_id"], result["product"])
                     self.conversation_context.set_last_action(f"human_feedback_create_product_{type(target_storage_for_action).__name__}",
                                                              {"input": tool_input, "result": result})

            except Exception as e:
                action_result_summary = f"Failed to create product '{name}' on {type(target_storage_for_action).__name__}: {str(e)}"
            print(f"[Human Feedback Action] {action_result_summary}")
            return action_result_summary

        print(f"[Human Feedback Action] {action_result_summary}")
        return action_result_summary


    def evaluate_responses(self, user_message: str, anthropic_response: str, openai_response: str, context_summary_for_eval: str, learnings_for_eval: str) -> Dict[str, Any]:
        print("\n--- Starting Evaluation by Gemini ---")
        try:
            # Initial evaluation prompt
            eval_prompt_parts = [
                f"User query: {user_message}",
                f"Current context provided to assistants:\n{context_summary_for_eval}",
                f"Anthropic Claude response:\n{anthropic_response}",
                f"OpenAI GPT response:\n{openai_response}",
                learnings_for_eval, # Relevant past learnings
                "Please evaluate both responses based on accuracy, efficiency, context awareness, and helpfulness. Provide an overall score (1-10) for each and detailed reasoning."
            ]
            eval_prompt = "\n\n".join(filter(None, eval_prompt_parts)) # filter(None, ...) removes empty strings
            # print(f"Gemini Eval Prompt (first 300 chars): {eval_prompt[:300]}...")

            gemini_response_obj = eval_model_instance.generate_content(eval_prompt)
            evaluation_text = gemini_response_obj.text
            print(f"Gemini Raw Initial Evaluation:\n{evaluation_text}")

            clarification_details = {"used": False, "needed": "", "provided_input": "", "action_summary": ""}

            # Check if Gemini needs clarification
            if "CLARIFICATION NEEDED:" in evaluation_text.upper():
                clarification_details["used"] = True
                clarification_details["needed"] = self.extract_clarification_needed(evaluation_text)
                print(f"--- Human Clarification Indicated by Evaluator ---")
                print(f"Clarification needed by evaluator: {clarification_details['needed']}")
                try:
                    human_input_for_eval = input(f"Enter human clarification for evaluator (or type 'skip' or 'quit'): ")
                    if human_input_for_eval.lower() in ['quit', 'exit', 'stop', 'q']:
                        print("Exiting the system. Goodbye!")
                        raise SystemExit("User requested exit during evaluation")

                    elif human_input_for_eval.lower() != 'skip':
                        clarification_details["provided_input"] = human_input_for_eval
                        target_storage_for_action = None
                        # Ask which store to apply data modification if any
                        if any(cmd in human_input_for_eval.lower() for cmd in ["update order", "create product"]): # Basic check
                            store_choice = input("Apply data change to (A)nthropic's store, (O)penAI's store, or (S)kip data change? [A/O/S]: ").lower()
                            if store_choice == 'a':
                                target_storage_for_action = self.anthropic_storage
                            elif store_choice == 'o':
                                target_storage_for_action = self.openai_storage

                        action_summary = self.process_human_feedback_actions(human_input_for_eval, target_storage_for_action)
                        clarification_details["action_summary"] = action_summary

                        # Store this interaction as a learning
                        self.store_learning(user_message, clarification_details["needed"], f"{human_input_for_eval} (Action: {action_summary})")

                        # Re-evaluate with clarification
                        updated_eval_prompt = f"{eval_prompt}\n\nHuman clarification provided to evaluator: {human_input_for_eval}\nAction taken based on feedback: {action_summary}\nPlease re-evaluate incorporating this."
                        updated_gemini_response = eval_model_instance.generate_content(updated_eval_prompt)
                        evaluation_text = updated_gemini_response.text
                        print(f"Gemini Raw Re-Evaluation after human input:\n{evaluation_text}")
                    else:
                        print("Skipping human input for evaluator.")
                        clarification_details["provided_input"] = "Skipped by user"
                except EOFError:
                    print("EOFError: Skipping human clarification for evaluator (non-interactive).")
                    clarification_details["provided_input"] = "Skipped (non-interactive)"


            # Sanity Check: Compare end-state dictionaries
            anthropic_data_state = {
                "customers": self.anthropic_storage.customers,
                "products": self.anthropic_storage.products,
                "orders": self.anthropic_storage.orders
            }
            openai_data_state = {
                "customers": self.openai_storage.customers,
                "products": self.openai_storage.products,
                "orders": self.openai_storage.orders
            }

            comparison_prompt_parts = [
                evaluation_text, # Include the previous evaluation
                "\n\n--- Data Store Comparison Task ---",
                "As a final step, please compare the following data store states from Anthropic and OpenAI.",
                f"Anthropic's Data Store State:\n{json.dumps(anthropic_data_state, indent=2, default=str)}",
                f"OpenAI's Data Store State:\n{json.dumps(openai_data_state, indent=2, default=str)}",
                "1. Identify any key differences between the two data stores.",
                "2. Explain plausible reasons for these differences based on the agents' actions during this turn (if known).",
                "3. State whether these differences, now explicitly reviewed, cause you to update your previous scores or assessment for either agent. If so, provide the updated scores and rationale."
            ]
            comparison_prompt = "\n\n".join(comparison_prompt_parts)
            # print(f"Gemini Comparison Prompt (first 300 chars): {comparison_prompt[:300]}...")

            comparison_response_obj = eval_model_instance.generate_content(comparison_prompt)
            final_evaluation_text = comparison_response_obj.text # This now includes the comparison
            print(f"Gemini Full Evaluation (including Data Store Comparison):\n{final_evaluation_text}")


            anthropic_score = self.extract_score(final_evaluation_text, "Anthropic")
            openai_score = self.extract_score(final_evaluation_text, "OpenAI")

            return {
                "anthropic_score": anthropic_score, "openai_score": openai_score,
                "full_evaluation": final_evaluation_text,
                "clarification_details": clarification_details
            }
        except Exception as e:
            print(f"Error in evaluation: {str(e)}")
            import traceback; traceback.print_exc()
            return {
                "error": f"Error in evaluation: {str(e)}",
                "anthropic_score": 0, "openai_score": 0,
                "full_evaluation": f"Evaluation failed: {str(e)}",
                "clarification_details": {"used": False, "action_summary": ""}
            }

    def extract_clarification_needed(self, evaluation_text: str) -> str:
        clarification_match = re.search(r"CLARIFICATION NEEDED:\\s*(.*?)(?:\\n|$)", evaluation_text, re.IGNORECASE | re.DOTALL)
        if clarification_match and clarification_match.group(1).strip():
            return clarification_match.group(1).strip()
        # Fallback if the specific format isn't found but the keyword is present
        lines = evaluation_text.splitlines()
        for i, line in enumerate(lines):
            if "CLARIFICATION NEEDED:" in line.upper():
                return line.split("CLARIFICATION NEEDED:", 1)[-1].strip() + "\n" + "\n".join(lines[i+1:i+3]) # return a bit more context
        return "Evaluator indicated clarification needed, but specific question not formatted as expected. Review raw evaluation."


    def store_learning(self, query: str, clarification_context: str, human_input: str):
        # Use a general key or derive keywords if needed
        learning_key = f"learning_for_query_like_{self.extract_keywords(query)[:2]}" # Simple key
        if not query and "General feedback" in clarification_context:
             learning_key = "general_user_feedback"


        timestamp = datetime.now().isoformat()
        log_entry = {
            "original_query_context": query, # Could be the user_message or context of clarification
            "evaluator_clarification_prompt_or_context": clarification_context, # What prompted this learning
            "human_input_or_learning": human_input,
            "timestamp": timestamp
        }

        if learning_key not in self.human_feedback_learnings:
            self.human_feedback_learnings[learning_key] = []
        self.human_feedback_learnings[learning_key].append(log_entry)
        print(f"Learning stored under key '{learning_key}': '{human_input[:100]}...'")


    def check_relevant_learnings(self, query: str) -> Optional[str]:
        relevant_learnings_text = []
        # Check for general feedback first
        if "general_user_feedback" in self.human_feedback_learnings:
            for learning in self.human_feedback_learnings["general_user_feedback"]:
                 relevant_learnings_text.append(
                    f"- General User Feedback (from {learning['timestamp']}): {learning['human_input_or_learning']}"
                )

        # Check for query-specific learnings
        keywords = self.extract_keywords(query)
        for keyword in keywords: # Check first 2 keywords for broader match
            learning_key_prefix = f"learning_for_query_like_{keyword}"
            for key, learnings_list in self.human_feedback_learnings.items():
                if key.startswith(learning_key_prefix) or keyword in key :
                    for learning in learnings_list:
                        # Avoid re-adding general feedback if already covered
                        if learning_key_prefix == "general_user_feedback" and "general_user_feedback" in self.human_feedback_learnings:
                            continue

                        text_to_add = (
                            f"- Context: '{learning['original_query_context']}' ({learning['timestamp']})\n"
                            f"  Situation: {learning['evaluator_clarification_prompt_or_context']}\n"
                            f"  User Input/Learning: {learning['human_input_or_learning']}"
                        )
                        if text_to_add not in relevant_learnings_text: # Avoid duplicates
                             relevant_learnings_text.append(text_to_add)

        return "\n\n".join(list(set(relevant_learnings_text))) if relevant_learnings_text else None # Use set to ensure unique learnings

    def extract_keywords(self, text: str) -> List[str]:
        if not text: return ["general"]
        words = re.findall(r'\b\w{4,}\b', text.lower())
        stop_words = {"the", "and", "is", "in", "to", "a", "of", "for", "with", "on", "at", "what", "how", "show", "tell", "please", "what's", "i'd", "like", "user", "query", "this", "that", "context"}
        extracted = list(set(word for word in words if word not in stop_words))
        return extracted if extracted else ["generic"]


    def extract_score(self, evaluation_text: str, model_name_pattern: str) -> int:
        # Try to find scores after the data comparison section if it exists
        comparison_section_start = evaluation_text.upper().rfind("--- DATA STORE COMPARISON TASK ---")
        search_text = evaluation_text
        if comparison_section_start != -1:
            # If comparison exists, and it might contain "updated scores", prioritize that part
            update_score_marker = re.search(r"update your previous scores|updated scores and rationale", evaluation_text[comparison_section_start:], re.IGNORECASE)
            if update_score_marker:
                search_text = evaluation_text[comparison_section_start:] # Search in the comparison part onwards

        patterns = [
            rf"{model_name_pattern}.*?Overall Score.*?(\d+)/10",
            rf"{model_name_pattern}.*?Overall Score:\s*(\d+)",
            rf"Overall Score.*?{model_name_pattern}.*?:\s*(\d+)",
            rf"{model_name_pattern}.*?score.*?:.*?(\d+)",
            rf"{model_name_pattern}.*?\bscore\b.*?(\d+)", # More general
            rf"{model_name_pattern}.*?rating.*?:.*?(\d+)",
            rf"{model_name_pattern}.*?\b(\\d+)/10",
             # Try to find updated scores first if mentioned
            rf"Updated score for {model_name_pattern}.*?:.*?(\d+)",
            rf"{model_name_pattern}.*?updated overall score.*?:.*?(\d+)"
        ]
        # Search in reverse for patterns to catch the latest score mention if text is long
        for p_str in reversed(patterns):
            # Find all matches and take the last one, as it's more likely to be the final/updated score
            matches = list(re.finditer(p_str, search_text, re.IGNORECASE | re.DOTALL))
            if matches:
                last_match = matches[-1]
                if last_match.group(1):
                    try:
                        score = int(last_match.group(1))
                        print(f"Extracted score {score} for '{model_name_pattern}' using pattern: {p_str}")
                        return score
                    except ValueError:
                        continue
        print(f"Could not extract score for '{model_name_pattern}' from eval text snippet (tried specific patterns):\n{search_text[-500:]}...") # Show tail
        return 0 # Default if no score found

print("DualAgentEvaluator class defined.")

DualAgentEvaluator class defined.


In [64]:
#Main Execution (or whichever cell contains your main() function)

""" Sample queries:
* Show me all the products available
* I'd like to order 25 Perplexinators, please
* Show me the status of my order
* (If the order is not in Shipped state, then) Please ship my order now
* How many Perplexinators are now left in stock?
* Add a new customer: Bill Leece, bill.leece@mail.com, +1.222.333.4444
* Add new new product: Gizmo X, description: A fancy gizmo, price: 29.99, inventory: 50
* Update Gizzmo's price to 99.99
* I need to update our insurance policy, so I need to know the total value of everything that I have in our inventory. Please tell me this amount.
* Summarize your learnings from our recent interactions.
"""


def main():
    print("\nStarting Main Execution...\n")
    # Basic check for placeholder API keys (if UserDataMock or similar was used)
    # Adjust this check if you're loading keys differently
    # if any('YOUR_KEY_HERE' in key_value for key_value in [ANTHROPIC_API_KEY, OPENAI_API_KEY, GOOGLE_API_KEY]):
    #     print("ERROR: Placeholder API keys detected. Update API key setup. Exiting.")
    #     return

    agent = DualAgentEvaluator()
    results_log = []

    while True:
        try:
            user_query = input("\nEnter your query (or 'quit', 'exit', 'stop', 'q' to end): ")
            if user_query.lower() in ['quit', 'exit', 'stop', 'q']:
                print("Exiting the system. Goodbye!")
                break

            if not user_query.strip():
                print("Empty query, please enter something.")
                continue

            result = agent.process_user_request(user_query)
            results_log.append(result)
        except SystemExit as se: # Catch SystemExit if user quits during clarification
            print(f"System exit requested: {se}")
            break
        except Exception as e:
            print(f"CRITICAL ERROR processing query '{user_query}': {e}")
            import traceback
            traceback.print_exc()
            results_log.append({
                "user_message": user_query, "anthropic_response": "ERROR", "openai_response": "ERROR",
                "evaluation": {"anthropic_score": 0, "openai_score": 0, "full_evaluation": f"Critical error: {e}",
                               "clarification_details": {"used": False, "action_summary": ""}}
            })

    print("\n\n===== EVALUATION SUMMARY =====")
    total_anthropic, total_openai, num_q = 0, 0, 0 # Initialize num_q

    for i, res in enumerate(results_log):
        if not res: # Skip if result is None or empty
            print(f"\nQuery {i+1}: Skipped (empty result).")
            continue

        num_q +=1 # Increment only for valid results
        print(f"\nQuery {i+1}: {res.get('user_message', 'N/A')}")
        print(f"  Anthropic Resp: {str(res.get('anthropic_response', 'N/A'))[:100]}...")
        print(f"  OpenAI Resp: {str(res.get('openai_response', 'N/A'))[:100]}...")

        eval_data = res.get('evaluation', {})
        anth_s = eval_data.get('anthropic_score',0)
        open_s = eval_data.get('openai_score',0)
        total_anthropic += anth_s
        total_openai += open_s
        print(f"  Scores - Anthropic: {anth_s}, OpenAI: {open_s}")

        clarif_details = eval_data.get('clarification_details',{})
        if clarif_details.get('used'):
            print(f"    Clarification: Needed='{clarif_details.get('needed', 'N/A')}', Provided='{clarif_details.get('provided_input', 'N/A')}'")
            if clarif_details.get('action_summary'):
                 print(f"    Action from Clarification: {clarif_details['action_summary']}")


        winner = "Tie"
        if anth_s is not None and open_s is not None: # Ensure scores are not None
            if anth_s > open_s: winner = "Anthropic"
            elif open_s > anth_s: winner = "OpenAI"
        print(f"  Query Winner: {winner}")


    print(f"\n----- Overall Performance -----")
    if num_q > 0:
        print(f"Avg Anthropic: {total_anthropic/num_q:.2f}, Avg OpenAI: {total_openai/num_q:.2f}")
    else:
        print("No queries processed to calculate average scores.")
    print(f"Total Anthropic: {total_anthropic}, Total OpenAI: {total_openai}")
    overall_winner = "Tie"
    if total_anthropic > total_openai: overall_winner = "Anthropic"
    elif total_openai > total_anthropic: overall_winner = "OpenAI"
    print(f"Overall Winner: {overall_winner}")

    if agent.human_feedback_learnings:
        print("\n----- Learned Clarifications / Feedback (for AI Models & Evaluator) -----")
        for kw, l_list in agent.human_feedback_learnings.items():
            print(f"Keyword/Context Key: {kw}")
            for item in l_list:
                # Corrected keys used here:
                q_context = item.get('original_query_context', 'N/A')
                eval_context = item.get('evaluator_clarification_prompt_or_context', 'N/A')
                learning_input = item.get('human_input_or_learning', 'N/A')
                timestamp = item.get('timestamp', 'N/A')
                print(f"  - Query Context: '{q_context}' (Timestamp: {timestamp})")
                print(f"    Evaluator Context/Feedback Type: {eval_context}")
                print(f"    Learning Input: {learning_input}")
    else:
        print("\nNo human feedback learnings were stored during this session.")
    print("\nExecution Finished.")

# if __name__ == "__main__": # This block is usually for script execution, not Colab cells
#     main()
# print("Main function defined. Call main() in a new cell to run the evaluation.")

In [65]:
main()


Starting Main Execution...

DualAgentEvaluator initialized with separate Storage for Anthropic and OpenAI.
DualAgentEvaluator initialized. OpenAI tools formatted (9 tools).

Enter your query (or 'quit', 'exit', 'stop', 'q' to end): I'd like to order 25 Perplexinators, please


User Message: I'd like to order 25 Perplexinators, please
Current Context Summary for Models:
No specific context items set yet.
------------------------------------------------------------

Anthropic API Call #1. System: ' You are a helpful customer service assistant for an e-comme...', Messages count: 1
Last message role: user
Anthropic Tool Call: get_product_info, Input: {'product_id_or_name': 'Perplexinator'}
--- [Tool Dispatcher] Attempting tool: get_product_info with input: {"product_id_or_name": "Perplexinator"} for storage: Storage ---
[Tool Helper] find_product_by_name: Matched 'Perplexinator' to 'Perplexinator' (ID: P3) with score 100 (in Storage)
[Tool Executed] get_product_info: Found by name (fuzzy)